<a href="https://colab.research.google.com/github/Lily999-ru/Preddiplomnaya_Praktika/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ИМПОРТИРОВАНИЕ БИБЛИОТЕК
from flask import Flask, render_template, request, send_file
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import os

# ЗАГРУЗКА МОДЕЛИ
MODEL_DIR = "./model"

# СОЗДАНИЕ FLASK-ПРИЛОЖЕНИЯ
app = Flask(__name__, static_folder="static", template_folder="templates")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Загрузка токенайзера и модели...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, local_files_only=True)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR, local_files_only=True)
model.to(device)
model.eval()
print("Модель загружена.")

# Словарь для преобразования числовых меток обратно в текстовые категории.
inv_map = {
    0: "Гипотезы и открытия",
    1: "Здравоохранение и экспертные мнения",
    2: "Общественное здоровье"
}

# ВЫЗОВ ФУНКЦИИ ПРЕДСКАЗАНИЯ
def predict_with_rejection(text, threshold=0.6, max_length=256):
    # Проверка корректности формата.
    if not isinstance(text, str) or not text.strip():
        return {"error": "Некорректный формат текста."}

    # Токенизация текста, преобразование в индексы.
    enc = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=max_length,
        return_tensors="pt"
    )

    # Перенос всех токенов на выбранное устройство (CPU или GPU).
    enc = {k: v.to(device) for k, v in enc.items()}

    # Отключение вычисления градиентов.
    with torch.no_grad():
        # Прямой проход модели.
        outputs = model(**enc)
        # Извлечение логитов.
        logits = outputs.logits
        # Перевод в вероятности.
        probs = F.softmax(logits, dim=1).cpu().numpy()[0]

    # Поиск наиболее вероятного класса.
    # Индекс класса.
    max_idx = int(np.argmax(probs))
    # Уверенность модели.
    max_prob = float(probs[max_idx])

    # Логика отклонения предсказания.
    # Если меньше порога, отклонение классификации.
    if max_prob < threshold:
        decision = "Статья не относится к медицине, или не достаточно уверенности для классификации."
    else:
        decision = "Классифицировано."

    # Возврат результатов предсказания.
    return {
        "label": inv_map[max_idx],
        "confidence": round(max_prob, 4),
        "decision": decision,
    }


# НАСТРОЙКА МАРШРУТОВ
# Главная страница
@app.route("/")
def index():
    return render_template("index.html")


# Страница загрузки CSV
@app.route("/csvupload", methods=["GET"])
def csvupload_page():
    return render_template("csvupload.html")

# Страница ручного ввода текста
@app.route("/textinput", methods=["GET"])
def textinput_page():
    return render_template("textinput.html")

# Настройка загрузки датасета как CSV файл
@app.route("/predict_csv", methods=["POST"])
def predict_csv():

    csv_file = request.files.get("csv_file")
    # Ошибки при не правильном формате файла.
    if not csv_file:
        return render_template("results.html", error="Вы не загрузили CSV файл.")
    try:
        df = pd.read_csv(csv_file)
    except Exception:
        return render_template("results.html", error="Ошибка чтения CSV файла, пожалуйста проверьте формат файла.")
    # Вывод ошибки,если в файле нет колонки с текстом.
    if "text" not in df.columns:
        return render_template("results.html", error="В CSV файле должна быть колонка 'text'.")
    # Создание таблицы для результатов.
    results = []
    for _, row in df.iterrows():
        text = str(row["text"])
        title = row.get("title", "—")
        short = text[:200] + "..." if len(text) > 200 else text

        pred = predict_with_rejection(text)
        results.append([title, short, pred["label"], pred["confidence"]])

        # Сохранение результатов в таблицу
        global last_results_df
        last_results_df = pd.DataFrame(results, columns=["title", "short", "label", "confidence"])

    # Возврат страницы результатов.
    return render_template("results.html", tables=results)

# Настройка загрузки датасета как ручной ввод
@app.route("/predict_text", methods=["POST"])
def predict_text():

    title = request.form.get("manual_title", "—")
    text = request.form.get("manual_text", "")
    # Проверка длины текста (Мининум 200 символов)
    if len(text.strip()) < 200:
        # Ошибка
        return render_template("results.html", error="Текст слишком короткий. Минимум 200 символов.")

    short = text[:200] + "..." if len(text) > 200 else text
    pred = predict_with_rejection(text)

    # Результаты
    results = [[title, short, pred["label"], pred["confidence"]]]

    # Сохранение результатов в таблицу
    global last_results_df
    last_results_df = pd.DataFrame(results, columns=["title", "short", "label", "confidence"])

    # Возврат страницы результатов.
    return render_template(
    "results.html",
    result=pred,
    title=title,
    short_text=short
)

# СОХРАНЕНИЕ РЕЗУЛЬТАТОВ
# Глобальная переменная для хранения последних результатов
last_results_df = None

# Настройка для скачивания результатов
@app.route("/download")
def download_file():

    global last_results_df
    # Ошибка
    if last_results_df is None:
        return "Нет данных для сохранения. Сначала выполните классификацию."
    # Название файла
    file_path = "classification_results.csv"
    last_results_df.to_csv(file_path, index=False, encoding="utf-8")
    # Возврат файла для скачивания
    return send_file(file_path, as_attachment=True)


# ЗАПУСК
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5001, debug=True)
